In [3]:
import sqlite3
import numpy as np
import pandas as pd
import json
import math
import DirectedGraph as DG
from IPython.display import clear_output

cnx = sqlite3.connect('ultimate_player_database.db')

In [4]:
def sql_table(conn):
   cursorObj = conn.cursor()
   cursorObj.execute("SELECT name FROM sqlite_master WHERE type='table';")
   return([k[0] for k in cursorObj.fetchall()])

tables = sql_table(cnx)

print(tables)

for k in tables: 
   print("TABLE: {tablename}".format(tablename = k))
   query = "SELECT * FROM {tablename} WHERE 1=0".format(tablename = k)
   print(pd.read_sql_query(query, cnx))
   count_query =  "SELECT COUNT(*) FROM {tablename}".format(tablename = k)
   print("COUNT OF {tablename}: {count}".format(tablename = k,count = pd.read_sql_query(count_query, cnx)['COUNT(*)'][0]))
   print("")

['tournament_info', 'sets', 'players', 'ranking', 'ranking_seasons']
TABLE: tournament_info
Empty DataFrame
Columns: [game, key, cleaned_name, source, tournament_name, tournament_event, season, rank, start, end, country, state, city, entrants, placings, losses, bracket_types, online, lat, lng]
Index: []
COUNT OF tournament_info: 98737

TABLE: sets
Empty DataFrame
Columns: [key, game, tournament_key, winner_id, p1_id, p2_id, p1_score, p2_score, location_names, bracket_name, bracket_order, set_order, best_of, game_data]
Index: []
COUNT OF sets: 7250884

TABLE: players
Empty DataFrame
Columns: [game, player_id, tag, all_tags, prefixes, social, country, state, region, c_country, c_state, c_region, placings, characters, alias]
Index: []
COUNT OF players: 487844

TABLE: ranking
Empty DataFrame
Columns: [game, ranking_name, priority, region, seasons, tournaments, icon]
Index: []
COUNT OF ranking: 1

TABLE: ranking_seasons
Empty DataFrame
Columns: [game, ranking_name, season, start, end, total

In [50]:
query = "SELECT game_data FROM sets WHERE game_data != '[]'"

df = pd.read_sql_query(query, cnx)

data = list()

for k in range(len(df)):

    if k % 50000 == 0:
        clear_output()
        print("{prop} % complete".format(prop = math.floor(100*k/len(df))))

    str = df.loc[k][0]

    while True:
        id1 = str.find("{")
        id2 = str.find("}")
        if (id1 != -1 ) and (id2 != -1):
            new_data = json.loads(str[id1:id2+1])
            data.append(new_data)
        else:
            break  
        str = str[id2+1:]

game_df = pd.DataFrame(data).fillna(-1)

game_df.to_csv("game_data.csv")

clear_output()
print("100 % complete")


100 % complete


In [3]:
# Reload data without re-computing:
# IMPORTANT: MUST RELOAD AFTER 1st PASS IN ORDER TO HAVE HOMOGENOUS DATA TYPES
game_df = pd.read_csv("game_data.csv",dtype={'winner_id' : "string", 'loser_id' : "string", 'winner_score' : "string", 'loser_score' : "string"})

In [93]:
players = set(game_df['winner_id']).difference({np.nan})
pivot = np.random.choice(tuple(players))
print(pivot)

i = 0

while True:
    sub_df = game_df.loc[(game_df['winner_id'] == pivot) or (game_df['loser_id'] == pivot)]
    player_subset = set(game_df.loc[game_df['winner_id'] == pivot]['loser_id']).union(set(game_df.loc[game_df['loser_id'] == pivot]['winner_id'])).union({pivot})

    n = len(player_subset)
    clear_output()
    print("Iteration: {it} \n Size of State Space: {size}\n".format(it = i, size = n))

    dg = DG.DirectedGraph().set_telep(0.01)

    k = 0

    for winner in player_subset:

        if k % np.round(n/50) == 0:
            print("{prop} % complete".format(prop = math.floor(100*k/n)))
        k += 1

        data_bits = sub_df.loc[game_df['winner_id'] == winner]

        for loser in data_bits['loser_id']:
            if loser not in player_subset:
                continue
            dg.incLink(winner,loser,add=True)

    stat = dg.stationary()
    sort_stat = sorted(stat, key = lambda x: stat[x],reverse=True)
    if pivot == sort_stat[1]:
        clear_output()
        print("Best Player: {p}".format(p = pivot))
        break
    pivot = sort_stat[1]
    i += 1

1667281


ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [81]:
print(len(dg.states))
dg.set_telep(0.01)
stat = dg.stationary()

1057


In [87]:
sort_stat = sorted(stat, key = lambda x: stat[x],reverse=True)
print(np.array([[k,stat[k]] for k in sort_stat]))
print(pivot)
print(sort_stat.index(pivot))
print(len(sort_stat))


[['539670' '0.02379322783535139']
 ['258371' '0.023669787857451854']
 ['152054' '0.023213158160436693']
 ...
 ['2052496' '9.51150442316902e-06']
 ['1966932' '9.51150442316902e-06']
 ['2041639' '9.51150442316902e-06']]
252680
11
1057
